In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geopy.distance import geodesic 
from statistics import mean 
import json
from datetime import datetime

In [2]:
sidewalk_to_pano = pd.read_csv('panos_for_fake_preds.csv')

In [3]:
sidewalk_to_pano.head()

,linkId,segmentId,whichEndpoint,panoId,headingDegree
0,37484,17748,start,2117,135
1,24291,2974,end,17585,45
2,8385,3223,end,3598,45
3,31412,19259,end,12030,135
4,50050,9123,end,8203,45


In [9]:
sidewalk_to_pano = sidewalk_to_pano.drop_duplicates(['panoId', 'headingDegree'])

In [10]:
sidewalk_to_pano.shape

(991, 5)

In [24]:
# for now just making all of these linearId's 0 
# I don't think we actually need those for this table
df = pd.DataFrame()
df['panoId'] = sidewalk_to_pano['panoId']
df['linearId'] = 0
df['headingDegree'] = sidewalk_to_pano['headingDegree']
df['label'] = 1
df['source'] = 'fake'
df['id'] = [i + 1 for i in df.index.values.tolist()]

In [25]:
df_to_load = df.filter(['id', 'panoId', 'linearId', 'headingDegree', 'label', 'source', 'updateTs'])

In [27]:
# this isn't necessary in future - was trying to bypass 
# weird thing in first row
df_to_load = df_to_load[1:]

In [28]:
import pymysql
connection = pymysql.connect(host='aa6f8tcntjfd0z.c5625ddefrth.us-west-2.rds.amazonaws.com',
                             port=3306,
                             user= # ask Emily :) 
                             password= # ask Emily :) 
                             db='ebdb')
cursor=connection.cursor()

In [29]:
# NOTE: this method is way slow
# creating column list for insertion
cols = "`,`".join([str(i) for i in df_to_load.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_to_load.iterrows():
    sql = "INSERT INTO `pano_feature` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()